In [ ]:
import sys
sys.path.insert(0, '../..')

In [ ]:
%load_ext autoreload
%autoreload 2
import edunn as nn
import numpy as np

# Clasificación con CNN

También podemos entrenar una red neuronal para clasificar las imágenes de dígitos escritos a mano del conjunto de datos de [MNIST](http://yann.lecun.com/exdb/mnist/). Este conjunto de datos es un clásico en el aprendizaje automático, y es un buen punto de partida para probar las redes neuronales convolucionales. Intentá probar agregando/quitando capas adicionales y variando los parámetros de las mismas.


In [ ]:
x,y,classes=nn.datasets.load_classification("mnist")
# normalización de los datos
x = (x-x.mean())/x.std()
n, din = x.shape
# calcular cantidad de clases
classes = y.max()+1
print("Tamaños de x e y:", x.shape,y.shape)
x.min(), x.max()

In [ ]:
i=np.random.randint(0, x.shape[0])

In [ ]:
import matplotlib.pyplot as plt
plt.title(y[i])
plt.imshow(np.reshape(x[i],(28,28)),cmap="gray")

In [ ]:
x = x.reshape(-1, 1, 28, 28)

In [ ]:
def calculate_in_features(input_size, layers):
    output_size = input_size
    for layer in layers:
        output_size = (output_size - layer['kernel_size'] + 2*layer['padding']) // layer['stride'] + 1
        out_channels = layer['out_channels']
    in_features = output_size * output_size * out_channels
    return in_features

layers = [
    {'out_channels': 32,    'kernel_size': 3,   'stride': 1, 'padding': 0},
    {'out_channels': 32,    'kernel_size': 2,   'stride': 2, 'padding': 0},
]
input_size = x.shape[-1]
in_features = calculate_in_features(input_size, layers)
in_features

In [ ]:
#Red convolucional
initializer = nn.initializers.KaimingNormal()#nn.initializers.RandomNormal(1e-20)
model = nn.Sequential([
    nn.Conv2d(in_channels=1,     out_channels=32,    kernel_size=(3,3),      stride=1, padding=0, kernel_initializer=initializer),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Dense(input_size=in_features,output_size=32,activation_name="relu",linear_initializer=None),
    nn.Dense(input_size=32,output_size=classes,activation_name="softmax",linear_initializer=None),
    ])

error = nn.MeanError(nn.CrossEntropyWithLabels())
optimizer = nn.GradientDescent(lr=0.1,epochs=10,batch_size=16)

In [ ]:
# np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Algoritmo de optimización
history = optimizer.optimize(model,x,y,error)
nn.plot.plot_history(history,error_name=error.name)

In [ ]:
print("Métricas del modelo:")
y_pred=model.forward(x)
y_pred_labels=nn.utils.onehot2labels(y_pred)
nn.metrics.classification_summary(y,y_pred_labels)

---

In [ ]:
x = x.squeeze().reshape(-1, 28 * 28)

In [ ]:
hidden_dim=32
#Red con dos capas 
model = nn.Sequential([nn.Dense(din,hidden_dim,activation_name="relu"),
                       nn.Dense(hidden_dim,classes,activation_name="softmax"),
                      ])

error = nn.MeanError(nn.CrossEntropyWithLabels())
optimizer = nn.GradientDescent(lr=0.01,epochs=100,batch_size=16)

In [ ]:
# Algoritmo de optimización
history = optimizer.optimize(model,x,y,error)
nn.plot.plot_history(history,error_name=error.name)

In [ ]:
print("Métricas del modelo:")
y_pred=model.forward(x)
y_pred_labels=nn.utils.onehot2labels(y_pred)
nn.metrics.classification_summary(y,y_pred_labels)